# Summary Generator


Generate a summary starting from all the previously created documentation file.

## 0. Imports

In [1]:
import os
from langchain_community.chat_models import ChatOpenAI
from langchain_community.document_loaders import DirectoryLoader, UnstructuredMarkdownLoader
from langchain_text_splitters import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain import hub
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain



## 1. Loading Files

In [2]:
path = "C:\\Users\\reply\\RepoAgent\\markdown_docs"
loader = DirectoryLoader(path, glob="./*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)
docs = loader.load()

0it [00:00, ?it/s]


In [3]:
def load_docs(path):
        all_docs = []
        abs = os.path.normpath(os.path.abspath(path))  # Normalize and convert root_path to an absolute path

        for subdir, _, _ in os.walk(abs):
            loader = DirectoryLoader(os.path.join(abs, subdir), glob="./*.md", show_progress=True, loader_cls=UnstructuredMarkdownLoader)    
            docs = loader.load()
            all_docs.extend(docs)
        return all_docs


In [4]:
llm = ChatOpenAI(temperature=0.1, model_name="gpt-3.5-turbo-1106")


C:\Users\reply\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
#import runnable sequence
from langchain_core.runnables import RunnableSequence

In [6]:
map_prompt = hub.pull("rlm/map-prompt")
reduce_prompt = hub.pull("rlm/reduce-prompt")
map_chain = LLMChain(llm=llm, prompt=map_prompt)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)




C:\Users\reply\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [7]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain_text_splitters import CharacterTextSplitter

llm = ChatOpenAI(temperature=0)

In [8]:

# Map Template
map_template = """The following is a set of documents:
{docs}
Based on this list of documents, please make a short description of their contents.
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce Template
reduce_template = """The following is a set of summaries:
{docs}
Please distill these summaries into a final, consolidated summary of the overall contents. Ensure the final summary captures the main points and key details from each document.
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)


In [9]:
# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=4000,
)

In [10]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

In [11]:
def split_documents(doc, chunk_size=250, chunk_overlap=30):
    """ Split a document into chunks of text."""

    headers_to_split_on = [
         ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]

    markdown_splitter = MarkdownHeaderTextSplitter(
        headers_to_split_on=headers_to_split_on, strip_headers=False
    )
    md_header_splits = markdown_splitter.split_text( doc.page_content)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap
    )

    # Split
    splits = text_splitter.split_documents(md_header_splits)
   
    return splits

In [12]:
docs = load_docs(path)

0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|██████████| 8/8 [00:00<00:00,  9.36it/s]


In [17]:
all_splits = []
for doc in docs:
        splits = split_documents(doc, chunk_size=10000, chunk_overlap=0)
        for doc_split in splits:
            filename = os.path.basename(list(doc.metadata.values())[0])
            doc_split.metadata = {'source':filename}        
        all_splits.extend(splits)

In [19]:
len(all_splits)

66

In [20]:
result = map_reduce_chain.invoke(all_splits)


In [22]:
result["output_text"]

'The set of documents provided offers a comprehensive guide for developers working on a Python codebase, covering functions, classes, and testing frameworks. The documents detail tasks such as generating markdown summaries, managing metadata information, handling exceptions, and updating documentation. They also describe classes like ChangeDetector, TaskManager, and ProjectManager, along with their methods and functionalities. Proper documentation, error handling, and code structure are emphasized for efficient project management, along with unit testing frameworks for ensuring code reliability. Overall, the documents provide insights into managing tasks, handling exceptions, and generating structured documentation effectively, with a focus on functionality, testing, and best practices.'

In [ ]:
def format_text(text):
    # Example formatting: Insert newlines after each period for demonstration
    formatted_text = a.replace(". ", ".\n")
    return formatted_text

In [ ]:
a = format_text(result["output_text"])

In [ ]:
#save a on a file
a.replace(". ", ".\n")
with open("output.txt", "w") as file:
    file.write(a)

In [ ]:
refine_template = """We have an existing summary that needs to be updated based on new information.
                        Existing Summary:
                        {existing_summary}

                        New Information:
                        {new_docs}

                        Please refine the existing summary to incorporate the new information accurately and comprehensively.
                        Helpful Answer:"""
refine_prompt = PromptTemplate.from_template(refine_template)
refine_chain = LLMChain(llm=llm, prompt=refine_prompt)


In [ ]:
refine_chain.invoke()

In [ ]:
all_splits

[]

In [ ]:
from langchain.chains.summarize import load_summarize_chain
chain2 = load_summarize_chain(llm, chain_type="refine")


In [ ]:
all_splits[0]

Document(page_content='FunctionDef main  \nmain: The function of main is to copy Markdown documentation files from a specified folder to a destination folder, create a README.md file if it does not exist, and organize the copied files accordingly.  \nparameters:\n- markdown_docs_folder: The folder containing the Markdown documentation files.\n- book_name: The name of the book being generated.\n- repo_path: The path to the repository.  \nCode Description:', metadata={'source': 'generate_repoagent_books.md'})

In [ ]:
#start time
import time
start = time.time()
res = chain2.invoke(docs)
end = time.time()
print(end - start)

In [ ]:
res

{'input_documents': [Document(page_content='FunctionDef main  \nmain: The function of main is to copy Markdown documentation files from a specified folder to a destination folder, create a README.md file if it does not exist, and organize the copied files accordingly.  \nparameters:\n- markdown_docs_folder: The folder containing the Markdown documentation files.\n- book_name: The name of the book being generated.\n- repo_path: The path to the repository.  \nCode Description:', metadata={'source': 'generate_repoagent_books.md'})],
 'output_text': 'The main function copies Markdown documentation files from a specified folder to a destination folder, creates a README.md file if it does not exist, and organizes the copied files. It takes parameters for the folder containing the files, the book name, and the repository path.'}